# 大数相乘

大数相乘就是数字特别大，相乘的结果会超出基本数据类型的表示范围，所以这样的数不能直接进行相乘。这种问题可以有三种解决办法：

## Karatsuba算法
karatsuba算法是一种快速相乘算法，采用分治递归的方法求解，此算法在1960年由Anatolii Alexeevitch Karatsuba 提出。普通相乘的复杂度为$O(n^2)$，而Karatsuba算法的复杂度仅为$O(n^{log_2{3}})$。下面来看一下这个算法的实现过程：

假设有两个数 $x = 12345$ 和 $y = 6789$，我们将这两个数分别拆成两部分，即:

$x = a + b$

$y = c + d$


其中 $a = 123 * 10^{n/2}$，$b = 45$，$c = 67 * 10^{n/2}$，$d = 89$，这里 $n = max(len(str(x)), len(str(y)))$
，于是
$$
\begin{aligned} & x * y \\=&\left(a * 10^{n / 2}+b\right) *\left(c * 10^{n / 2}+d\right) \\=& a c * 10^{n}+(a d+b c) * 10^{n / 2}+b d \\=& a c * 10^{n}+[(a+b) *(c+d)-a c-b d] * 10^{n / 2}+b d \end{aligned}
$$
其中，$(a d+b c) = (a+b) *(c+d)-a c-b d$，所以我们可以利用递归，每次只需要每次计算$a c$，$(a+b) *(c+d)$ 和 $b d$，这样四次乘法运算就变成了三次(和六次加减法)。因此复杂度为：
$$T(n)=3 T\left(\frac{n}{2}\right)+6 n=O\left(n^{\log _{2} 3}\right)$$
下面看一个例子，这里指数取为 $n//2$，于是 $a=123$，$b=45$，$c=67$，$a=89$ ：
$$
\begin{aligned} 12345 &=123 \cdot 100+45 \\ 6789 &=67 \cdot 100+89 \end{aligned}
$$
那么 $a*c$，$(a+b) *(c+d)$ 和 $a*c$ 分别为：
$$
\begin{aligned} z_{0} &=a * c=123 \times 67=8241 \\ z_{1} &=b * d=45 \times 89=4005 \\ z_{2} &=((a+b) *(c+d)-a * c-b * d) \\ &=(123+45) \times(67+89)-z_{0}-z_{1}=26208-8241-4005=13962 \end{aligned}
$$
最终结果为：
$$
\begin{array}{l}{\text {ans}=z_{0} \cdot 10^{2 * 2}+z_{2} \cdot 10^{2}+z_{1}} \\ {\text {ans}=8241 \cdot 10^{4}+13962 \cdot 10^{2}+4005=83810205}
\end{array}
$$
至此，我们可以写出整个过程如下：

1. 首先根据两个数的长度进行截取， $n = max(len(str(x)), len(str(y)))$，那么取 $m = n//2$ 作为指数。
2. 计算$a * c$，$(a+b) *(c+d)$ 和 $b * d$
3. 递归计算$a * c$，$(a+b) *(c+d)$ 和 $b * d$
4. 设置递归终止条件

```python
if len(str(x))==1 or len(str(y))==1:
    return x*y
```
下面是具体Python实现。

In [8]:
def karatsubaMul(x, y):
    '''
    Karatsuba算法计算大数相乘
    '''
    if len(str(x)) == 1 or len(str(y)) == 1:
        return x * y
    n = max(len(str(x)), len(str(y)))
    exp = n // 2
    
    a = x // 10**exp
    b = x % 10**exp
    c = y // 10**exp
    d = y % 10**exp
    ac = karatsubaMul(a, c)
    bd = karatsubaMul(b, d)
    abcd = karatsubaMul(a+b, c+d)
    adbc = abcd - ac - bd
    return ac * 10**(2*exp) + adbc * 10**exp + bd

print(karatsubaMul(1234567, 123), 1234567*123)

151851741 151851741



## 模拟乘法手算的累加算法

<div align=center><img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/big_data_mul.png" width="500" /><div/>

模拟手算就像小学的时候计算乘法那样，先拿一个数分别与另一个数的个位、十位、百位、、、依次相乘，然后再错位相加得到计算结果。下面的程序就是按照这个想法实现的。每个变量的含义如下：

$num1:$ 字符串，大数1

$num2:$ 字符串，大数2

$result:$ 列表，存放每个位上的计算结果

$alist:$ 列表，存放num1的每一位整数

$blist:$ 列表，存放num2的每一位整数

$bit\_result:$ 存放按位计算的结果

$tens:$ 存放按位计算的结果的进位数


In [19]:
def mulByBit(num1, num2):
    alist = list(map(int, reversed(num1))) # 将字符串反转并转换成整数列表
    blist = list(map(int, reversed(num2)))
    result = [0]*(len(alist) + len(blist))
    for i, vala in enumerate(alist):
        tens = 0
        for j, valb in enumerate(blist):
            bit_result = vala * valb + tens + result[i+j]
            result[i+j] = bit_result % 10
            tens = bit_result // 10
        result[i+j+1] = tens
    result = ''.join(list(map(str, result[::-1])))
    return result if result[0] != '0' else result[1:]

# test
a = '1234567'
b = '123'
print(mulByBit(a, b), int(a)*int(b))

151851741 151851741


## 模拟乘法手算的改进算法

In [10]:
def mulByBit(num1, num2):
    if num1[0] == '0' or num2[0] == '0': return '0'
    alist = list(map(int, num1))
    blist = list(map(int, num2))
    result = [0]*(len(alist) + len(blist))
    for i in range(len(alist)-1, -1, -1): # 个位索引是高位
        for j in range(len(blist)-1, -1, -1):
            result[i+j] += (alist[i] * blist[j] + result[i+j+1]) // 10 # 索引小是数字的高位
            result[i+j+1] = (alist[i] * blist[j] + result[i+j+1]) % 10 # 索引大是数字的低位
    result = ''.join(map(str, result))
    return  result if result[0] != '0' else result[1:]

# test
a = '0'
b = '456'
print(mulByBit(a, b), int(a)*int(b))

0 0


In [38]:
def bitMul(num1, num2):
    alist = list(map(int, num1))
    blist = list(map(int, num2))
    result = [0]*(len(alist) + len(blist))
    for i in range(len(alist)-1, -1, -1):
        for j in range(len(blist)-1, -1, -1):
            result[i+j] += (alist[i] * blist[j] + result[i+j+1]) // 10
            result[i+j+1] = (alist[i] * blist[j] + result[i+j+1]) % 10
    result = ''.join(map(str, result))
    return result if result[0] != '0' else result[1:]

sts = input().strip().split()
print(sts[0], sts[1])
print(bitMul(sts[0], sts[1]))

123 456
123 456
56088


# 最大乘积

给定一个无序数组，包含正数、负数和0，要求从中找出3个数的乘积，使得乘积最大，要求时间复杂度：$O(n)$，空间复杂度：$O(1)$。

输入描述:

无序整数数组A[n]

输出描述:

满足条件的最大乘积

输入例子:

4

3 4 1 2

输出例子:

24

**解决思路:**

考虑到数组中既有正数，又有负数和 $0$ ，且要求出最大乘积，于是分情况讨论如下，其中max1, max2, max2, min1, min2分别为最大的三个数和最小的两个数：
1. 全是正数：则结果应为最大的三个数相乘，即 result = max1 * max2 * max3
2. 全是负数：则结果应为最大的三个负数相乘，即 result = max1 * max2 * max3，和情况1相同
3. 有正有负：这种情况下，结果有两种可能，即 result = min1 * min2 * max1 or result = max1 * max2 * max3

所以，综上可知，result = max(max1 * max2 * max3, min1 * min2 * max1)

In [36]:
def max_three_product(arr):
    min1 = float('inf')
    min2 = float('inf')
    max1 = float('-inf')
    max2 = float('-inf')
    max3 = float('-inf')
    for val in arr:
        if val < min1:
            min1, min2 = val, min1
        elif val < min2:
            min2 = val
        if max1 < val:
            max1, max2, max3 = val, max1, max2
        elif max2 < val:
            max2, max3 = val, max2
        elif max3 < val:
            max3 = val

    return max(max1*max2*max3, min1*min2*max1)

n = input()
arr = list(map(int, input().split()))
print(max_three_product(arr))

6
-1 0 -2 3 -5 7
70


# 拼多多扑克牌游戏

In [21]:
def backtrack(i, n, result, s1, s2, curr):
    if i == n:
        if curr == s2:
            print(" ".join(result))
        return
    else:
        backtrack(i+1, n, result+"d", s1[1:], s2, curr)
        backtrack(i+1, n, result+"l", s1[1:], s2, s1[0]+curr)
        backtrack(i+1, n, result+"r", s1[1:], s2, curr+s1[0])
    
n = int(input())
while n > 0:
    st1 = input()
    st2 = input()
    size = len(st1)
    print("{")
    backtrack(0, size, "", st1, st2, "")
    print("}")
    n -= 1

3
123
3
{
d d l
d d r
}
123
321
{
l l l
r l l
}
12
23
{
}


# 瓜子二手车最大数字

In [44]:
def backtrack(result, curr, arr):
    if len(arr) == 0:
        result.append(curr[:])
        return
    else:
        for i in range(len(arr)):
            backtrack(result, curr+arr[i], arr[0:i]+arr[i+1:])

arr = ['1', '201', '20', '9', '8']
result = []
backtrack(result, '', arr)
print(max(list(map(int, result))))

98202011


In [49]:
from functools import cmp_to_key

class Solution():
    def compare(self, num1, num2):
        t = str(num1) + str(num2)
        s = str(num2) + str(num1)
        if t < s:
            return 1
        elif t > s:
            return -1
        else:
            return 0
        
    def PrintMinNumber(self, numbers):
        if numbers == None:
            return ""
        lens = len(numbers)
        if len(numbers) == 0:
            return ""
        result = sorted(numbers, key = cmp_to_key(self.compare))
        return "".join((str(x) for x in result))
    
arr = ['1', '201', '20', '9', '8']
print(Solution().PrintMinNumber(arr))

98202011


# 华为买钉子

钉子数 1 <= n <= 200，钉子只有两种包装，一种一盒 4 个， 一种一盒 9 个，不能零卖，求最小的盒数，刚好买够 n 个钉子，如果没有输出 -1。

In [6]:
n = int(input())

dp = [-1] * 201
dp[4] = 1
dp[8] = 2
dp[9] = 1
for i in range(10, 201):
    if dp[i-4] > 0 and dp[i-9] > 0:
        dp[i] = min(dp[i-4], dp[i-9]) + 1
    elif dp[i-4] > 0 and dp[i-9] < 0:
        dp[i] = dp[i-4] + 1
    elif dp[i-4] < 0 and dp[i-9] > 0:
        dp[i] = dp[i-9] + 1
    else:
        dp[i] = -1
print(dp[n])

12
3


# 字节跳动-第一个缺失的数

In [27]:
def firstMissing(num):
    size = len(num)
    for i in range(len(num)):
        while 1 <= num[i] and num[i] <= size and num[i] != num[num[i] - 1]:
            num[num[i] - 1], num[i] = num[i], num[num[i] - 1]
            
    for i in range(len(num)):
        if i + 1 != num[i]:
            return i + 1
    return size + 1

num = [1, 8, 9, 11, 12]
print(firstMissing(num))

2


# 爱奇艺排列

In [26]:
def Permutation(curr, num, a, count):
    if len(num) == 0:
        flag = False
        for i in range(len(a)):
            if a[i] == 0:
                if curr[i] < curr[i+1]:
                    flag = True
                else:
                    flag = False
                    break
            else:
                if curr[i] > curr[i+1]:
                    flag = True
                else:
                    flag = False
                    break
        if flag:
            count[0] = count[0] + 1
    else:
        for i in range(len(num)):
            curr.append(num[i])
            Permutation(curr, num[:i] + num[i+1:], a, count)
            curr.pop()

num = int(input())
a = list(map(int, input().split()))
arr = list(range(1, num+1))
count = []
count.append(0)
Permutation([], arr, a, count)
print(count[0])

4
1 1 0
3


# 美团-删除重复字符

aaabbb -> "no"

ababab -> ababab

aabbcd -> cd

In [32]:
s = input()

i, j = 0, 0
size = len(s)
result = []
while i < size and j < size: # 双指针，都从头开始移动
    if s[i] == s[j]: # 若两者一直相等，只移动 j
        j += 1
    else:
        if j == 1: # 对应于 ab 的情况，即开头两个字符就不相同
            result.append(s[i])
        result.append(s[j]) # 对应于 i 和 j 位置的字符不相同，将 s[j] 存入
        i = j # 将 i 移到 j 的位置
        j += 1 # j 前进一位
        if j < size and s[i] == s[j]: # 如果 s[i] == s[j]，将上一个存入的 pop 出来
            result.pop()
            
if result:
    print("".join(result))
else:
    print("no")

aaabbb
no


In [40]:
# 方法二：
s = input()

size = len(s)
result = ""
i = 0
duplicated = False
while i < size:
    while i + 1 < size and s[i] == s[i+1]:
        duplicated = True
        i += 1
    if duplicated:
        i += 1
    else:
        result += s[i]
        i += 1
    duplicated = False
if result:
    print(result)
else:
    print("no")

aaabccd
bd


# 美团-寻找最小子字符串

In [79]:
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        result = s
        pl, pr, match = 0, 0, 0
        start, minlen = 0, float("inf")
        size = len(s)
        window = {}
        need = {}
        for i in range(len(t)):
            need[t[i]] = need[t[i]] + 1 if t[i] in need else 1
        while pr < size:
            c1 = s[pr]
            if c1 in need:
                window[c1] = window[c1] + 1 if c1 in window else 1
                if window[c1] == need[c1]:
                    match += 1
            pr += 1
            
            while match == len(need):
                if pr - pl < minlen:
                    start = pl
                    minlen = pr - pl
                c2 = s[pl]
                if c2 in need:
                    window[c2] -= 1
                    if window[c2] < need[c2]:
                        match -= 1
                pl += 1
                
        return "" if minlen == float("inf") else s[start:start+minlen]

# 三七互娱-股票收入最高

股票价格：arr = [7, 1, 5, 3, 6, 4]

收入最高：arr[4] - arr[1] = 6 - 1 = 5

In [78]:
def maxprofit(price):
    size = len(price)
    minval = price[0]
    dp = [0] * size
    for i in range(1, size):
        if price[i] < minval:
            minval = price[i]
        dp[i] = max(dp[i-1], price[i] - minval)
    return dp[size-1]

price = [7, 1, 5, 3, 6, 4]
print(maxprofit(price))

5


# 360-散步

<img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/360-散步.png" width="700" />

**思路：**

设置两个长度为 n+1 的 dp1 和 dp2 数组，dp[i] 表示某次停下是否能到第 i 个位置，dp[i] = 0 表示不能到达，dp[i] = 1 表示能到达：

1. 初始化 dp1[1...n] = 1，表示可以从任意位置开始
2. 对每走 Dj 距离，方向可以向左或向右，那么走完 Dj 距离后，可到达的位置用 dp2 表示，状态转移方程为：

``` python
if dp1[j] == 1 and j + d[i] <= n:
    dp2[j + d[i]] = 1
if dp1[j] == 1 and j - d[i] >= 1:
    dp2[j - d[i]] = 1
```
3. dp1 = dp2，重复步骤2。

对于每走一步 $Di$ 都要对 dp[j] 进行一次遍历，总共 N * M 次，所以时间复杂度为 $O(N*M)$，空间复杂度为 $O(N)$

In [74]:
def numOfEnd(n, m, d):
    dp1 = [1] * (n+1)
    dp2 = [0] * (n+1)

    for i in range(m):
        for j in range(1, n+1):
            dp2[j] = 0
        for j in range(1, n+1):
            if dp1[j] == 1 and j + d[i] <= n:
                dp2[j + d[i]] = 1
            if dp1[j] == 1 and j - d[i] >= 1:
                dp2[j - d[i]] = 1
        dp1 = dp2[:]
    
    num = 0
    for i in range(1, n+1):
        if dp2[i] == 1:
            num += 1
    return num

# n, m = map(int, input().split())
# d = []
# for i in range(m):
#     d.append(int(input()))
n, m = 10, 3
d = [5, 2, 6]
result = numOfEnd(n, m ,d)
print(result)

8


# 咪咕-计算输入日期是一年的第几天

In [80]:
arr = list(map(int, input().split()))

mon = [0] * 13
for i in range(1, 13):
    if i in [1, 3, 5, 7, 8, 10, 12]:
        mon[i] = 31
    elif i == 2:
        if (arr[0]%4==0 and arr[0]%100!=0) or arr[0]%400==0:
            mon[i]  = 29
        else:
            mon[i] = 28
    else:
        mon[i] = 30

day = 0
for i in range(1, arr[1]):
    day = day + mon[i]
day = day + arr[2]
print(day)

2019 2 2
33


# 快手

## 第一题

<img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/快手1.jpg" width="300" />

**思路：**

分别找出所有坐标点中的 xmin, xmax, ymin, ymax，然后输出：xmin ymin,xmin ymax,xmax ymax,xmax ymin.

In [1]:
inputs = input().split(",")
pts = []
for st in inputs:
    pts.append(list(map(int, st.split())))
    
zips = list(zip(*pts))
xmin = min(zips[0])
xmax = max(zips[0])
ymin = min(zips[1])
ymax = max(zips[1])
result = []
result.append(str(xmin) + " " + str(ymin))
result.append(str(xmin) + " " + str(ymax))
result.append(str(xmax) + " " + str(ymax))
result.append(str(xmax) + " " + str(ymin))
print(",".join(result))

0 1,2 0,2 3
0 0,0 3,2 3,2 0


## 第二题

<img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/快手2.jpg" width="300" />

**思路：**

先看三个例子：

    1 5 2 3

        i

    3 3 2 5

        i
        
当遇到 nums[i] < nums[i-1]时，根据 nums[i] 与 nums[i-2] 的大小关系，有两种情况：
1. nums[i] > nums[i-2]，此时令 nums[i-1] = nums[i]
2. nums[i] < nums[i-2]，此时令 nums[i] = nums[i-1]

还有一种特殊情况，如下：就是 i = 1 是第二个元素下标，其不存在 nums[i-2]，此时只需使 nums[i-1] = nums[i]

        5 2 3 4

          i
          
因为题目要求：最多改变 1 个元素的情况下，该数组能否变成一个非递减数列。所以要设置一个 flag = 1，当修改一次值之后，使 flag = 0，一旦当 falg = 0 时，还存在 nums[i] < nums[i-1]，就返回 False。

In [7]:
def checkPossibility(nums):
    size = len(nums)
    flag = 1
    for i in range(1, size):
        if nums[i] < nums[i-1]:
            if flag == 0: return False
            if i == 1 or nums[i] >= nums[i-2]:
                nums[i-1] = nums[i]
            else:
                nums[i] = nums[i-1] 
            flag -= 1
    return True


arr = list(map(int, input().split(",")))
print(nonDecrease(arr))

4,5,2,9,10
1


## 第三题

<img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/快手3.jpg" width="300" />

**思路：**

In [9]:
def longestPalindrome(s):
    size = len(s)
    maxlens, result = 0, ""
    dp = [[False] * size for i in range(size)]
    for lens in range(size):
        for i in range(size-lens):
            j = i + lens
            if lens < 2:
                dp[i][j] = s[i] == s[j]
            else:
                dp[i][j] = dp[i+1][j-1] and s[i] == s[j]
            if dp[i][j] and maxlens < lens + 1:
                maxlens = lens + 1
                result = s[i:j+1]
    return result
    
s = input()
print(longestPalindrome(s))

ababc
aba


## 第四题

<img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/快手4.jpg" width="300" />

**思路：**

In [10]:
def convertZ(s, numRows):
    if numRows < 2: return s
    size = len(s)
    result = ["" for _ in range(size)]
    i, flag = 0, -1
    for char in s:
        result[i] += char
        if i == 0 or i == numRows-1:
            flag = -flag
        i += flag
    return "".join(result)

inputs = input().split(",")
st, n = inputs[0], int(inputs[1])
print(convertZ(st, n))

LEETCODEISHIRING,3
LCIRETOESIIGEDHN


# 商汤

## 第一题

<img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/商汤1.jpg" width="300" />

In [11]:
class Solution:
    def uniquePaths(self, m: int, n: int) -> int:
        dp = [[0] * n for i in range(m)]
        for i in range(m):
            dp[i][0] = 1
        for i in range(n):
            dp[0][i] = 1
        for i in range(1, m):
            for j in range(1, n):
                dp[i][j] = dp[i-1][j] + dp[i][j-1]
        return dp[m-1][n-1]

## 第二题

<img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/商汤2.jpg" width="300" />

In [42]:
from typing import List
class Solution:
    def maxSubarraySumCircular(self, A: List[int]) -> int:
        size = len(A)
        nums = A + A
        result, q = nums[0], [0]
        s = [0] * 2 * size
        for i in range(1, 2*size):
            s[i] = s[i-1] + nums[i-1]
            while q and q[0] + size < i:
                q.pop(0)
            result = max(result, s[i] - s[q[0]])
            while q and s[q[-1]] > s[i]:
                q.pop()
            q.append(i)
        return result

# n = int(input())
# arr = list(map(int, input().split()))
n = 6
arr = [3, -1, 8, 6, 5, 2]
print(Solution().maxSubarraySumCircular(arr))


24


In [14]:
from typing import List
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        size = len(nums)
        arr = nums + nums
        dp = [0] * 2 * size # dp 存储的是当前 [0...i] 的子序列最大和
        dp[0] = nums[0]
        for i in range(1, 2 * size):
            dp[i] = max(dp[i-1] + arr[i], arr[i])
        
        maxsum = float("-inf")
        for i in range(2 * size):
            if dp[i] > maxsum:
                maxsum = dp[i]
        return maxsum
    
n = 6
nums = [3, -1, 8, 6, 5, 2]
print(Solution().maxSubArray(nums))

46


## 第三题

<img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/商汤3.jpg" width="300" />

## 第四题

<!img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/商汤4.jpg" width="300" />

# vivo

## 第一题

<img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/vivo1.jpg" width="300" />

In [34]:
class Solution:
    def jump(self, nums: List[int]) -> int:
        # 方法一：动态规划，超时
        # size = len(nums)
        # dp = [float("inf")] * size
        # dp[0] = 0
        # for i in range(1, size):
        #     for j in range(i):
        #         if nums[j] >= i - j:
        #             dp[i] = min(dp[i], dp[j] + 1) # dp[i] 表示到 i 位置的最少步数
        # return dp[-1]
        
        # 方法二：
        # 思路：我们需要寻找的是当前位置所能到达的范围内的某个最大的数组元素值，而不是当前元素所能跳的最大距离，如：
        # [3, 2, 5, 3, 1, 1, 1, 1]，当前位置下标 0 表示的值为 3，其表示的范围内包含了 [2, 5, 3]，此时我们应
        # 该选取的值为 max(2 + 1, 5 + 2, 3 + 3)，即 max(nums[i] + i)
        # 1. 我们用 curr_max 表示目前所能达到的最远坐标，如对第一个元素来说 curr_max = 0 + nums[0] = 3
        # 2. 用 next_max 表示当前元素(nums[0])所能到达的范围内，选取的吓一跳所能到达的最远距离，即：
        # next_max = max(2 + 1, 5 + 2, 3 + 3) = 7
        # 3. 遍历数组，当 i == curr_len 时，step += 1
        # 4. 当 next_max >= size - 1 时，return step + 1
        size = len(nums)
        if size < 2:
            return 0 # nums 长度为 0 或 1
        
        curr_max, next_max, step = 0, 0, 0
        for i in range(size):
            next_max = max(next_max, nums[i] + i)
            if next_max <= i: # 此条件可判断最后一个元素是否可达
                return -1
            if next_max >= size - 1:
                return step + 1
            if i == curr_max:
                step += 1
                curr_max = next_max
        

step_list = [2, 3, 1, 1, 0, 4]
step_list = [2, 1, 0, 4]
print(Solution().jump(step_list))

-1


## 第二题

<img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/vivo2.jpg" width="300" />

In [16]:
def solution(N,M):
    start, result = 0, []
    arr = list(range(1, N+1))
    while True:
        if len(arr) == 0:
            break
        start = (start + M - 1) % N
        result.append(arr.pop(start))
        N -= 1
    return " ".join(map(str, result))

N,M = [int(i) for i in input().split()]
print(solution(N, M))

6 3
3 6 4 2 5 1


## 第三题

<img src="http://github.com/lupanlpb/data_stru_and_algo/raw/master/images/vivo3.jpg" width="300" />